In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
def ducumentos_recebidos(cpi):
  url = "https://legis.senado.leg.br/comissoes/docsRecCPI?codcol={}".format(cpi)
  request_result = requests.get(url)
  soup = BeautifulSoup(request_result.text, "html.parser") 
  table = soup.find('table', attrs={'class':'table'})
  table_rows = table.find_all('tr')
  rows = []

  for tr in table_rows[1:]:
    td = tr.find_all('td')
    li = tr.find_all('li')   # Arquivos
    div = tr.find_all('div') # Em resposta
    row = [tr.text.replace('\n', '') for tr in td]
    
    arquivos = {}
    for i in range(len(li)):

      if li[i].a is not None:
        arquivos[li[i].text] = li[i].a['href']
      else:  
        arquivos[li[i].text] = ''

    emresposta = {}
    for i in range(len(div)):
      if div[i].a is not None:
        emresposta[div[i].a.text] = div[i].a['href']


    row[1] = arquivos
    row[7] = emresposta
    rows.append(row)

  df = pd.DataFrame(rows)
  df.columns = ['id', 'arquivos', 'data', 'remetente', 'origem', 'descricao', 'caixa', 'emresposta']
  df['id'] = df['id'].astype(int)
  df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
  df.set_index(['id'], inplace=True)
  
  return df

def oficios_expedidos(cpi):
  url = "https://legis.senado.leg.br/comissoes/docsEnvCPI?codcol={}".format(cpi)
  request_result = requests.get(url)
  soup = BeautifulSoup(request_result.text, "html.parser") 
  table = soup.find('table', attrs={'class':'table'})
  table_rows = table.find_all('tr')
  rows = []
  for tr in table_rows[1:]:
    td = tr.find_all('td')
    div = tr.find_all('div')
    row = [tr.text.replace('\n', '') for tr in td]

    requerimentos = {}
    for i in range(len(div)):
      if div[i].a is not None:
        requerimentos[div[i].a.text] = div[i].a['href']

    row[3] = requerimentos
    rows.append(row)

  df = pd.DataFrame(rows)
  df.columns = ['id', 'data', 'destinatario', 'requerimentos', 'ementa', 'observacao']
  df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
  df.set_index(['id'], inplace=True)
  
  return df

def ducumentos_apreciados(cpi):
  url = "https://legis.senado.leg.br/comissoes/reqsCPI?0&codcol={}&aprc=true&prej_retir=false&susp=false".format(cpi)
  request_result = requests.get(url)
  soup = BeautifulSoup(request_result.text, "html.parser") 
  table = soup.find('table', attrs={'class':'table'})
  table_rows = table.find_all('tr')
  rows = []
  for tr in table_rows[1:]:
    td = tr.find_all('td')
    row = [tr.text.replace('\n', '') for tr in td]
    rows.append(row)

  df = pd.DataFrame(rows)
  df.columns = ['id', 'data', 'ementa', 'autoria', 'situacao']
  #df['id'] = df['id'].astype(int)
  #df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
  df.set_index(['id'], inplace=True)
  
  return df

In [ ]:
# 2292 - Fake News
# 2441 - CPI Pandemia
cpi = 2441
df_expedidos = oficios_expedidos(cpi)
df_recebidos = ducumentos_recebidos(cpi)
#df_apreciados = ducumentos_apreciados(cpi)

In [ ]:
df_recebidos.sort_values('data', ascending=False).head()

,arquivos,data,remetente,origem,descricao,caixa,emresposta
id,,,,,,,
1403,"{'DOC 1401 parte 1': 'https://legis.senado.leg.br/sdleg-getter/documento/download/a295055b-bc63-4bdb-887b-e3ce7e43af19', 'DOC 1401 parte 2': 'https://legis.senado.leg.br/sdleg-getter/documento/download/2ee1978f-29db-4e4b-af6f-aa374c5e1c58'}",2021-07-02,Hospital Federal de Ipanema,OFÍCIO Nº 494/2021/HFI/RJ/SEMS/SE/MS02/07/2021,Encaminha as informações e documentação requisitadas ao Hospital Federal de Ipanema.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t,0,{'895 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/8b5ce507-948a-4b18-9d30-b5e038e79b79'}
1342,{'Doc 1342': ''},2021-07-02,RFB - Receita Federal do Brasil,"267/2021 – RFB/Copes/Diaes,30/06/2021","Transferência de Sigilo Fiscal de Instituto de Atenção Básica e Avançada à Saúde (IABAS) - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'873 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/2f5031b7-29b8-4c67-8567-56571f932d39'}
1351,{'Doc 1351': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 276/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Aloha International Comercio de Cosmeticos Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,"{'919 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/5da0e682-4f63-411a-80d9-9ab6a72eb58a', '920 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/425db6a2-14e2-4d7b-9a4b-48d963ca9f61', '921 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/c8ced010-cab1-402e-91af-40ee640707a4'}"
1350,{'Doc 1350': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 275/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Smartkids Brasil-Livros e Consultoria Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'918 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/dc7feb98-6e4d-416d-ae97-8626e512210f'}
1348,{'Doc 1348': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 273/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Central Brasileira Participacoes Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'916 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/21d2abdf-9c83-4f5a-9235-653fb158d40c'}


In [ ]:
df_expedidos[df_expedidos['ementa'].str.contains('igilo')]

,data,destinatario,requerimentos,ementa,observacao
id,,,,,
559 / 2021,2021-05-07,PGR - Procuradoria Geral da República,{},Cadastro de Servidor da Secretaria da CPIPANDEMIA para Acesso a Documentos Sigilosos no MPF-Drive.,
1104 / 2021,2021-05-19,MS - Ministério da Saúde,{},Solicitação de cadastro de servidor da Secretaria da CPIPANDEMIA para acesso a documentos sigilosos no SEI.,
1122 / 2021,2021-05-21,PRODASEN - Secretaria de Tecnologia da Informação do Senado Federal,{},Solicita envio dos logs de acesso aos documentos sigilosos da Comissão.,
1231 / 2021,2021-06-14,Prodasen Senado Federal,{},Dificuldade no Acesso a Documentos Sigilosos da CPIP ANDEMIA.,
1239 / 2021,2021-06-14,Banco Central do Brasil - Bacen,{'791 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/e7965ee8-490c-4f63-bb95-c3277e8554ec'},Transferência de Sigilo Bancário - ASSOCIAÇÃO DIGNIDADE MÉDICA DE PERNAMBUCO - ADM/PE,
...,...,...,...,...,...
1786 / 2021,2021-07-01,ANATEL - Agência Nacional de Telecomunicações,{'1038 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/058669c6-5366-4c0a-8831-3489269d0b1c'},Transferência de Sigilo Telefônico - LÍGIA NARA ARNAUD TOMAZ,
1787 / 2021,2021-07-01,ANATEL - Agência Nacional de Telecomunicações,{'1039 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/de3c905d-3968-419d-bedb-94cfc257ace9'},Transferência de Sigilo Telefônico - ALLAN LOPES DOS SANTOS,
1788 / 2021,2021-07-01,ANATEL - Agência Nacional de Telecomunicações,{'1040 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/cca48085-540f-4760-a7b5-c595e92f33d2'},Transferência de Sigilo Telefônico - TERCIO ARNAUD TOMAZ,


In [ ]:
df_recebidos[df_recebidos['remetente'] == 'RFB - Receita Federal do Brasil'].sort_values('data', ascending=False).head(10)

,arquivos,data,remetente,origem,descricao,caixa,emresposta
id,,,,,,,
1340,{'Doc 1340': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 264/2021 – RFB/Copes/Diaes02/07/2021,"Transferência de Sigilo Fiscal de Associação Filantrópica Nova Esperança - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'870 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/4adf39cd-7d67-45ae-85f0-0da488fbffbf'}
1341,{'Doc 1341': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 266/2021 – RFB/Copes/Diaes02/07/2021,"Transferência de Sigilo Fiscal de Instituto dos Lagos Rio - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'872 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/fe4c9c6a-c499-4537-b284-f444a96a6ca3'}
1353,{'Doc 1353': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 278/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Vip Xviii - Empreendimentos e Participacoes Ltda. - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'923 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/c4590f97-df97-490d-b8ef-2164365dd5c5'}
1352,{'Doc 1352': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 277/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Cwmv Sistema de Escolas Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t6 folha(s).\t\t\t\t\t\t\t",0,{'922 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/fc4c9f19-55e5-4460-abdd-252174edc6d4'}
1351,{'Doc 1351': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 276/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Aloha International Comercio de Cosmeticos Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,"{'919 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/5da0e682-4f63-411a-80d9-9ab6a72eb58a', '920 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/425db6a2-14e2-4d7b-9a4b-48d963ca9f61', '921 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/c8ced010-cab1-402e-91af-40ee640707a4'}"
1350,{'Doc 1350': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 275/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Smartkids Brasil-Livros e Consultoria Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'918 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/dc7feb98-6e4d-416d-ae97-8626e512210f'}
1349,{'Doc 1349': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 274/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Orion Consultoria Empreendimentos e Participacoes Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'917 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/973f2d9f-f480-4c9c-85ba-976c74a2828e'}
1348,{'Doc 1348': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 273/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Central Brasileira Participacoes Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,{'916 / 2021 ': 'https://legis.senado.leg.br/sdleg-getter/documento/download/21d2abdf-9c83-4f5a-9235-653fb158d40c'}
1347,{'Doc 1347': ''},2021-07-02,RFB - Receita Federal do Brasil,Not

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1370 entries, 1 to 1403
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   arquivos    1370 non-null   object        
 1   data        1370 non-null   datetime64[ns]
 2   remetente   1370 non-null   object        
 3   origem      1370 non-null   object        
 4   descricao   1370 non-null   object        
 5   caixa       1370 non-null   object        
 6   emresposta  1370 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 85.6+ KB


In [ ]:
pd.options.display.max_colwidth = 0

In [ ]:
df.sort_values('data', ascending=False).head()

,arquivos,data,remetente,origem,descricao,caixa,emresposta
id,,,,,,,
1403,"{'DOC 1401 parte 1': 'https://legis.senado.leg.br/sdleg-getter/documento/download/a295055b-bc63-4bdb-887b-e3ce7e43af19', 'DOC 1401 parte 2': 'https://legis.senado.leg.br/sdleg-getter/documento/download/2ee1978f-29db-4e4b-af6f-aa374c5e1c58'}",2021-07-02,Hospital Federal de Ipanema,OFÍCIO Nº 494/2021/HFI/RJ/SEMS/SE/MS02/07/2021,Encaminha as informações e documentação requisitadas ao Hospital Federal de Ipanema.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t,0,\t\t\t\t\t\t\tOf. 1612 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 895 / 2021 )\t\t\t\t\t\t\t
1342,{'Doc 1342': ''},2021-07-02,RFB - Receita Federal do Brasil,"267/2021 – RFB/Copes/Diaes,30/06/2021","Transferência de Sigilo Fiscal de Instituto de Atenção Básica e Avançada à Saúde (IABAS) - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,\t\t\t\t\t\t\tOf. 1651 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 873 / 2021 )\t\t\t\t\t\t\t
1351,{'Doc 1351': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 276/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Aloha International Comercio de Cosmeticos Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,\t\t\t\t\t\t\tOf. 1660 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 919 / 2021 )\t\t\t\t\t\t\t\t\t\t\t\t\t\tOf. 1661 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 920 / 2021 )\t\t\t\t\t\t\t\t\t\t\t\t\t\tOf. 1662 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 921 / 2021 )\t\t\t\t\t\t\t
1350,{'Doc 1350': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 275/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Smartkids Brasil-Livros e Consultoria Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,\t\t\t\t\t\t\tOf. 1659 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 918 / 2021 )\t\t\t\t\t\t\t
1348,{'Doc 1348': ''},2021-07-02,RFB - Receita Federal do Brasil,Nota nº 273/2021 – RFB/Copes/Diaes30/06/2021,"Transferência de Sigilo Fiscal de Central Brasileira Participacoes Ltda - Acesso Restrito: Sigilo Fiscal - Lei 5.172/1966, art. 198.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t folha(s).\t\t\t\t\t\t\t",0,\t\t\t\t\t\t\tOf. 1657 / 2021\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(Req. 916 / 2021 )\t\t\t\t\t\t\t


In [ ]:
df.remetente.value_counts().head(50)

MS - Ministério da Saúde                                 50
STF - Supremo Tribunal Federal                           41
RFB - Receita Federal do Brasil                          30
Procuradoria da República em Mato Grosso - Diamantino    25
Casa Civil da Presidência da República                   17
Tribunal de Contas da União - TCU                        17
BB - Banco do Brasil S.A.                                16
Governo do Estado do Ceará                               13
MRE - Ministério das  Relações  Exteriores               13
Claro S.A.                                               12
Governo do Distrito Federal                              12
Secretaria Estadual de Saúde de Alagoas                  12
Prefeitura de Manaus                                     11
Governo do Estado do Rio Grande do Norte                 11
Secretaria Municipal de Saúde de Manaus                  10
MD - Ministério da Defesa                                10
Governo do Estado do Rio de Janeiro     

In [ ]:
df[df['remetente'].str.contains('zuell')]

,arquivos,data,remetente,origem,descricao,caixa,emresposta
id,,,,,,,
2,{'DOC 002': 'https://legis.senado.leg.br/sdleg-getter/documento/download/4af93ab9-e154-4c40-85fb-ef4c7387e95d'},2021-05-04,General Eduardo Pazuello,DIEx nº 57 - Gab/SGEx EB: 64691.010609/202104/05/2021,"Informa que dois servidores do Poder Executivo Federal com que estava em contato apresentaram laudo positivo para Covid-19, e apresenta propostas para atender a convocação da CPI.\t\t\t\t\t\t\tDocumentação contendo:\t\t\t\t\t\t\t\t\t\t\t\t\t\t2 folha(s).\t\t\t\t\t\t\t",1,
